<h1> AO* Search Algorithm</h1>

<h2>Pseudocode</h2>

<pre>
Initialization:
    Set up initial graph with heuristic costs
    Set terminal nodes as solved.
    Set up and_edges
    Set MAX(FUTILITY) = 1000

Call aostar on head node
Repeat until head is solved or number of iterations >MAX{
    aostarUtil(head)
}

Function aostarUtil
    #aostarUtil
    if head is not visited{
        #explore_head
        For each and-arc or or-arc in from head{
            if or-arc{
            cost = h(node in arc) + EDGE cost
            } else if and-arc{}
            calculate cumulative cost of nodes from arc + EDGE costs
            }
        update head cost as minimum cost among the and-arc's and or-arc's
        If head is solved, track the path 
        }   
    }else 
    {
        #find_best_move
        find the best and-arc or or-arc from head node
        Call aostarUtil recursively for each node in best and-arc or from node in or-arc
        
        #check update
        Propagate the change of cost in the head node
        
    }
</pre>

<a href = 'https://drive.google.com/file/d/1RPXanmRIWqcnWTfZaVZQgg2LikY9s9De/view?usp=sharing'> heuristic map</a>

In [1]:
class Node:
    def __init__(self, index,cost,visited=False,is_solved=False,
                 and_map=False, or_map=False):
        self.index=index
        self.cost=cost
        self.visited=visited
        self.is_solved=is_solved
        self.and_map= and_map; self.or_map = or_map
        self.children=None
        self.path = False
        
    def __str__(self):
        return f'{self.index}: {self.cost}'
    
    def set_children(self,ch):
        self.children=ch
        

In [2]:
#Use case 1
adj=[]
n_nodes = 10
#heuristic costs
cost=[None,0,6,12,9,5,7,1,4,4,1]
children = {1:[2,3,4], 2:[5,6], 3:[7], 4:[8,9], 5:[10]}

adj = [Node(i, cost[i]) for i in range(n_nodes+1)]
for p,ch in children.items():
    ch_nodes = [adj[c] for c in ch]
    adj[p].set_children(ch_nodes)

and_edges=[]

ae = [adj[1],(adj[2],adj[3])]
and_edges.append(ae)
adj[2].and_map=True; adj[3].and_map = True
ae = [adj[4],(adj[8],adj[9])]
and_edges.append(ae)
adj[8].and_map=True; adj[9].and_map = True

for a in adj:
    if not a.children: a.is_solved=True #set for all terminal nodes
    if a.and_map==False: a.or_map=True
    #print(f'{a.index} and {a.and_map} or {a.or_map}')

In [711]:
""" #use case 2
adj=[]

n_nodes = 21
#heuristic costs
cost=[None,0,40,2,4,1,2,3,50,60,70,80,4,5,8,9,6,7,90,90,90,90]
children = {1:[2,3,4], 2:[5,6,7], 3:[8,9], 4:[10,11], 5:[12,13], 6:[14,15], 
           7:[16,17], 8:[18], 9:[19], 10:[20], 11:[21] }
#Note: the first node is dummy node
adj = [Node(i, cost[i]) for i in range(n_nodes+1)]
for p,ch in children.items():
    ch_nodes = [adj[c] for c in ch]
    adj[p].set_children(ch_nodes)

and_edges=[]

and_edges.append([adj[1], (adj[3], adj[4])])
adj[3].and_map = adj[4].and_map = True

and_edges.append([adj[2],(adj[5],adj[6], adj[7])])
adj[5].and_map = adj[6].and_map = adj[7].and_map=True

for a in adj:
    if not a.children: a.is_solved=True
    if a.and_map==False: a.or_map=True
    #print(f'{a.index} and {a.and_map} or {a.or_map} solved {a.is_solved}') """

In [3]:
#function to get all sibling nodes of a node part of an and-arc
def get_andarc(c):
    for idx, ae in and_edges:
        if c in ae : return ae

In [4]:
def explore_head(head):
    print(f'Head: {head.index}, Cost: {head.cost}')
    head.path=True
    head.visited=True; temp_cost = MAX; temp_map=[]
    #check if head has any and edges
    for c in head.children:
        if c in temp_map: continue;
        if c.and_map: # if the child is in the and edge
            ae = get_andarc(c) 
            cc=sum(aek.cost+EDGE for aek in ae)
            temp_solved = all([aek.is_solved for aek in ae])
 
            if temp_solved: 
                head.is_solved=True;
                if cc<temp_cost: 
                    bestsolved=ae;
            
            temp_cost = min(temp_cost,cc)

        else: # else if child is in the or edge
            if c.is_solved: 
                head.is_solved=True 
                cc = c.cost+EDGE
                if cc < temp_cost: 
                    bestsolved = [c]
            temp_cost = min(temp_cost,c.cost+EDGE)
                
   #head is explored now update the best value of head i.e. temp_cost  
    if temp_cost < MAX:
        head.cost=temp_cost
        print(f'Updated head {head.index} cost {head.cost} is_solved {head.is_solved}')
 
    #set up path
    if head.is_solved:
        for b in bestsolved: b.path=True
        for c in head.children: 
            if c not in bestsolved: c.path=False
    

In [5]:
def find_best_move(head):
     #find the best move
        print(f'##Find best move in {head.index}')
        bestCost=MAX;bestMove=None; bestAndArc=None
        temp_map=[]
        for c in head.children:
            if c in temp_map:continue
            if c.or_map:
                if bestCost>c.cost+EDGE:
                    bestCost = c.cost+EDGE
                    bestMove=c; 
            else: # and map
                ae = get_andarc(c)
                temp_map.extend(ae)
                cc=sum(aek.cost+EDGE for aek in ae)
                if bestCost>cc and cc!=0:
                    bestCost = cc; bestAndArc = ae; bestMove = c
            
        print(f'\n Moving forward, finding the best move,i>> {bestMove.index} and_map {bestMove.and_map} cost: {bestCost}')
        if bestMove.and_map:
            for ae in bestAndArc:aostarUtil(ae); 
        else:
            aostarUtil(bestMove); 


In [6]:
def check_update(head):
    temp_cost=MAX; temp_map=[]
    print(f'^^Check update {head.index}')
    for c in head.children:
        if c in temp_map:continue
        if c.or_map:
            if c.is_solved: head.is_solved=True
            cc = c.cost+EDGE
            temp_cost= min(temp_cost, c.cost+EDGE)
        elif c.and_map:
            f=True;cc=0
            ae = get_andarc(c)
            temp_map.extend(ae)
            temp_solved = all([aek.is_solved for aek in ae])
            cc=sum(aek.cost+EDGE for aek in ae)
            temp_cost = min(temp_cost,cc)
            
            if temp_solved : head.is_solved = True

    if temp_cost<=MAX: head.cost = temp_cost
    print(f'Propagate updated Cost of node {head.index} {head.cost} ')



In [7]:
path=[]
def aostarUtil(head):
    print(f'aostarUtil on {head.index}')
    if head.visited ==False:
        explore_head(head)  
    else:
        find_best_move(head)
        #check if any of the options were solved
        #if there are multiple solved nodes , select the best out of them 
        #also update the current cost i.e. head cost while backtracking to the root 
        check_update(head)

In [8]:
def aostar(head):
    iter = 0
    while head.is_solved==False and iter <MAX:
        print(f'\n  **Iteration {iter}')
        aostarUtil(head)
        iter+=1
    
    print('\n Updated costs')
    for a in adj[1:]:
        print(a.index,': ',a.cost, a.is_solved)

In [9]:
MAX=1000
EDGE=5 #g cost of edge, change to 5 for use case 2
aostar(adj[1]) 


  **Iteration 0
aostarUtil on 1
Head: 1, Cost: 0
Updated head 1 cost 14 is_solved False

  **Iteration 1
aostarUtil on 1
##Find best move in 1

 Moving forward, finding the best move,i>> 4 and_map False cost: 14
aostarUtil on 4
Head: 4, Cost: 9
Updated head 4 cost 18 is_solved True
^^Check update 1
Propagate updated Cost of node 1 23 

 Updated costs
1 :  23 True
2 :  6 False
3 :  12 False
4 :  18 True
5 :  5 False
6 :  7 True
7 :  1 True
8 :  4 True
9 :  4 True
10 :  1 True


In [10]:
print(f'Cost of Solution : {adj[1].cost}')
print('Path')
for a in adj:
    if a.path: print(f'{a.index} cost: {a.cost}')

Cost of Solution : 23
Path
1 cost: 23
4 cost: 18
8 cost: 4
9 cost: 4
